In [1]:
import os

os.chdir('/content/drive/MyDrive/PANPP')
!pip install -r requirement.txt

In [2]:
import shutil


os.chdir('/content/drive/MyDrive/PANPP/data')

if False:
    # Reset Training & Test Data
    print('Reset Training & Test Data')
    try:
        shutil.rmtree('TrainingData')
    except:
        pass
    try:
        shutil.rmtree('TestData')
    except:
        pass
    os.mkdir('TrainingData')
    os.mkdir('TestData')
    os.chdir('TrainingData')
    os.mkdir('image')
    os.mkdir('txt')
    os.chdir('../TestData')
    os.mkdir('image')
    os.mkdir('txt')
    os.chdir('..')

In [3]:
import csv
import numpy as np
import pandas as pd

from tqdm import tqdm
import cv2


def saveData(target, brand, img):
    if target == 'Train':
        targetDir = '/content/drive/MyDrive/PANPP/data/TrainingData'
    elif target == 'Test':
        targetDir = '/content/drive/MyDrive/PANPP/data/TestData'
    absDataDir = '/content/drive/MyDrive/PANPP/data/FlickrLogos-v2'
    # Make Target Data: IMG
    absImgDir = absDataDir + '/classes/jpg/' + brand + '/' + img
    shutil.copy(absImgDir, targetDir + '/image/' + img)
    # Make Target Data: Bbox (GT)
    absBboxDir = absDataDir + '/classes/masks/' + brand.lower() + '/' + img + '.bboxes.txt'
    bbox = pd.read_csv(absBboxDir, sep=' ')
    with open(targetDir + '/txt/' + img[:-4] + '.txt', 'a', encoding='utf-8') as f:
        wr = csv.writer(f)
        for i in range(len(bbox)):
            x, y, width, height = bbox.iloc[i]
            pos = (x + width // 2, y + height // 2)
            length = (width, height)
            deg = 0
            tmp = cv2.boxPoints((pos, length, deg))
            poly = np.array(tmp).astype(np.int32).reshape((-1))
            wr.writerow(poly)
    
os.chdir('/content/drive/MyDrive/PANPP/data/FlickrLogos-v2/classes/jpg')

cnt = 0
for brand in tqdm(os.listdir()):
    if not '.' in brand and not brand == 'no-logo':
        os.chdir('/content/drive/MyDrive/PANPP/data/FlickrLogos-v2/classes/jpg/' + brand)
        for img in tqdm(os.listdir()):
            if '.jpg' in img:
                if cnt % 4 == 0:
                    saveData('Test', brand, img)
                else:
                    saveData('Train', brand, img)
                cnt += 1

In [4]:
os.chdir('/content/drive/MyDrive/PANPP/data/TrainingData/image')
print('Training Data: No. Images', len(os.listdir()))

os.chdir('/content/drive/MyDrive/PANPP/data/TrainingData/txt')
print('Training Data: No. GT', len(os.listdir()))

os.chdir('/content/drive/MyDrive/PANPP/data/TestData/image')
print('Test Data: No. Images', len(os.listdir()))

os.chdir('/content/drive/MyDrive/PANPP/data/TestData/txt')
print('Test Data: No. GT', len(os.listdir()))

Training Data: No. Images 1680
Training Data: No. GT 1680
Test Data: No. Images 560
Test Data: No. GT 560


In [5]:
os.chdir('/content/drive/MyDrive/PANPP')

In [6]:
!sh train.sh

In [7]:
!sh test.sh